In [6]:
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense


In [7]:
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.12.0


In [8]:
train_dir = '/Users/sathvikchowdaryveerapaneni/Desktop/ANIE/DataSet/train'
test_dir = '/Users/sathvikchowdaryveerapaneni/Desktop/ANIE/DataSet/test'




In [11]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

Found 2004 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
